In [1]:
from bokeh.sampledata.autompg import autompg_clean as df
import hvplot.pandas
import pandas as pd
import panel as pn
import panel.widgets as pnw
import holoviews as hv
import pandas as pd
import geopandas as gpd
import json
import matplotlib as mpl
import pylab as plt
import numpy as np
from scipy import stats
from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.palettes import Category20c, Category20
from bokeh.plotting import figure
from math import pi
from bokeh.transform import cumsum


hv.extension('bokeh')
pn.extension('tabulator')

In [2]:
df = pd.read_csv('survey.csv')

df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [3]:
df.columns.values

array(['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence',
       'coworkers', 'supervisor', 'mental_health_interview',
       'phys_health_interview', 'mental_vs_physical', 'obs_consequence',
       'comments'], dtype=object)

In [23]:
df['seek_help'].value_counts()

seek_help
No            646
Don't know    363
Yes           250
Name: count, dtype: int64

In [24]:
df['mental_health_consequence'].value_counts()

mental_health_consequence
No       490
Maybe    477
Yes      292
Name: count, dtype: int64

In [4]:
df['treatment'].value_counts()

treatment
Yes    637
No     622
Name: count, dtype: int64

In [5]:
df['remote_work'].value_counts()

remote_work
No     883
Yes    376
Name: count, dtype: int64

In [6]:

# CPT_subset_merge[~(np.abs(stats.zscore(CPT_subset_merge[['Booked - Actual']])) < 3).all(axis=1)]
country_age = df.groupby(['Country'])['Age'].mean().to_frame().reset_index().sort_values(by='Age').tail(10)
country_age = country_age[(df['Age'] > 0) & (df['Age'] < 160)]
country_age = country_age[(np.abs(stats.zscore(country_age[['Age']])) < 3).all(axis=1)]

/var/folders/_q/s0ffv1256wx21ww07t4l8jn80000gn/T/ipykernel_52136/1312596375.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  country_age = country_age[(df['Age'] > 0) & (df['Age'] < 160)]


In [7]:
bar_chart = country_age.hvplot.bar(x='Country', y='Age', xlabel='Country', ylabel='Mean Age', rot=90, height=400, width=1000, title='Mean Age of Respondents by Country')
bar_chart

:Bars   [Country]   (Age)

In [8]:
country_count = df.groupby(['Country'])['Timestamp'].count().to_frame().reset_index().sort_values(by='Timestamp').tail(10)
country_count['color'] = Category20c[len(country_count)]
country_count['angle'] = country_count['Timestamp']/country_count['Timestamp'].sum() * 2*pi
country_count

,Country,Timestamp,color,angle
29,New Zealand,8,#3182bd,0.043370
20,India,10,#6baed6,0.054212
15,France,13,#9ecae1,0.070476
0,Australia,21,#c6dbef,0.113845
28,Netherlands,27,#e6550d,0.146373
21,Ireland,27,#fd8d3c,0.146373
17,Germany,45,#fdae6b,0.243955
7,Canada,72,#fdd0a2,0.390327
44,United Kingdom,185,#31a354,1.002924
45,United States,751,#74c476,4.071331


In [9]:
p = figure(height=350, title="Top 10 Country Count", toolbar_location=None,
           tools="hover", tooltips="@Country: @Timestamp", x_range=(-0.5, 1.0))

r = p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='Country', source=country_count)

p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None

bokeh_pane = pn.pane.Bokeh(p, theme="dark_minimal")
bokeh_pane

Bokeh(figure, theme='dark_minimal')

In [10]:
state_count = df.groupby(['state'])['Timestamp'].count().to_frame().reset_index().sort_values(by='Timestamp').tail(10)
state_count['color'] = Category20c[len(state_count)]
state_count['angle'] = state_count['Timestamp']/state_count['Timestamp'].sum() * 2*pi
state_count

,state,Timestamp,color,angle
11,IN,27,#3182bd,0.340655
31,OR,29,#6baed6,0.365888
32,PA,29,#9ecae1,0.365888
10,IL,29,#c6dbef,0.365888
29,OH,30,#e6550d,0.378505
37,TX,44,#fd8d3c,0.555141
36,TN,45,#fdae6b,0.567758
28,NY,57,#fdd0a2,0.719160
41,WA,70,#31a354,0.883179
2,CA,138,#74c476,1.741124


In [11]:
p2 = figure(height=350, title="Top 10 State Count", toolbar_location=None,
           tools="hover", tooltips="@state: @Timestamp", x_range=(-0.5, 1.0))

r2 = p2.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='state', source=state_count)

p2.axis.axis_label=None
p2.axis.visible=False
p2.grid.grid_line_color = None

bokeh_pane2 = pn.pane.Bokeh(p2, theme="dark_minimal")
bokeh_pane2

Bokeh(figure, theme='dark_minimal')

In [67]:
custom_order = ['1-5', '6-25', '26-100', '100-500', '500-1000', 'More than 1000']
df['no_employees'] = pd.Categorical(df['no_employees'], categories=custom_order, ordered=True)
leave_vs_num_employees = df.groupby(['no_employees', 'leave']).size().reset_index(name='count')

leave_vs_num_employees['no_employees'] = pd.Categorical(leave_vs_num_employees['no_employees'], categories=custom_order, ordered=True)

plot = leave_vs_num_employees.hvplot.bar(x='no_employees', y='count', by='leave', height=400, width=800, legend='top', xlabel='Number of Employees', ylabel='Count', rot=45, title='Count of Leave Policies by Number of Employees')

# panel = pn.Column("# Leave Policy Distribution by Company Size", plot)
# panel

# leave_vs_num_employees
plot

/var/folders/_q/s0ffv1256wx21ww07t4l8jn80000gn/T/ipykernel_52136/1994689842.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  leave_vs_num_employees = df.groupby(['no_employees', 'leave']).size().reset_index(name='count')
/Users/sunil/miniconda3/envs/data_science_test2/lib/python3.9/site-packages/holoviews/core/data/pandas.py:239: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dataset.data.groupby(group_by, sort=False)]


:Bars   [no_employees,leave]   (count)

In [13]:
treatment_vs_family_history = df.groupby(['treatment', 'family_history']).size().reset_index(name='count')

plot2 = treatment_vs_family_history.hvplot.bar(x='family_history', y='count', by='treatment', height=400, width=800, legend='top', xlabel='Family History', ylabel='Count', rot=45, title='Count of Treatment by Family History')

# panel = pn.Column("# Leave Policy Distribution by Company Size", plot)
# panel

plot2

:Bars   [family_history,treatment]   (count)

In [14]:
df['Gender'] = df['Gender'].replace('male', 'Male')
df['Gender'] = df['Gender'].replace('female', 'Female')
df['Gender'] = df['Gender'].replace('M', 'Male')
df['Gender'] = df['Gender'].replace('F', 'Female')
df['Gender'] = df['Gender'].replace('f', 'Female')
df['Gender'] = df['Gender'].replace('m', 'Male')
# df['Gender'].replace('male', 'Male')

df.loc[df['Gender'].str.contains('woman'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('Woman'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('female'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('Female'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains(' Female'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.startswith('Male'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains(' Male'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('msle'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('male '), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains(' Man'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('Malr'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('Mail'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('Mal'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains(' male'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('maile'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('Femake'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('femail'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('/she/'), 'Gender'] = 'Female'
df.loc[df['Gender'].str.contains('Man'), 'Gender'] = 'Male'
df.loc[df['Gender'].str.contains('Guy '), 'Gender'] = 'Male'
df.loc[~df['Gender'].isin(['Male', 'Female']), 'Gender'] = 'Male'



df['Gender'].value_counts()

Gender
Male      1007
Female     252
Name: count, dtype: int64

In [16]:
yes_treatment = df[(df['treatment'] == 'Yes') & (df['Age'] > 0) & (df['Age'] < 160)][['Age', 'remote_work']]
yes_treatment = yes_treatment[(np.abs(stats.zscore(yes_treatment[['Age']])) < 3).all(axis=1)]

# violin_plot = yes_treatment.hvplot.violin(y='Age', by='Gender', width=400, height=400, xlabel='Gender', ylabel='Age', title='Age Distribution Among Respondents Reporting Mental Health Conditions')

# violin_plot
 # * yes_treatment.hvplot.scatter(y='Age', x='remote_work', c='orange').opts(jitter=0.5)
box_plot = yes_treatment.hvplot.box(y='Age', by='remote_work', width = 1000, height = 400, title='Age Distribution vs. Gender')
box_plot = box_plot * yes_treatment.hvplot.scatter(y='Age', x='remote_work', c='orange').opts(jitter=0.5)
box_plot


:Overlay
   .BoxWhisker.I :BoxWhisker   [remote_work]   (Age)
   .Scatter.I    :Scatter   [remote_work]   (Age)

In [54]:
# grouped_bar = df.groupby(['remote_work', 'work_interfere']).size().unstack().hvplot.bar(stacked=False, height=400, width=800, xlabel='Remote Work', ylabel='Count', rot=45, title='Relationship Between Work Interference and Remote Work')

# grouped_bar


work_interfere_vs_remote = df.groupby(['remote_work', 'work_interfere']).size().unstack().reset_index()
work_interfere_vs_remote['total'] = work_interfere_vs_remote.drop(columns=['remote_work']).sum(axis=1)
work_interfere_vs_remote['Never'] = work_interfere_vs_remote['Never']/work_interfere_vs_remote['total']
work_interfere_vs_remote['Often'] = work_interfere_vs_remote['Often']/work_interfere_vs_remote['total']
work_interfere_vs_remote['Rarely'] = work_interfere_vs_remote['Rarely']/work_interfere_vs_remote['total']
work_interfere_vs_remote['Sometimes'] = work_interfere_vs_remote['Sometimes']/work_interfere_vs_remote['total']

long_format_df = work_interfere_vs_remote.melt(id_vars=['remote_work'], value_vars=['Never', 'Often', 'Rarely', 'Sometimes'], var_name='Work Interference', value_name='Proportion')

work_interfere_vs_remote_bar = long_format_df.hvplot.bar(x='remote_work', y='Proportion', by='Work Interference', stacked=False, height=400, width=800, xlabel='Remote Work', ylabel='Proportion', rot=45, title='Proportional Relationship Between Work Interference and Remote Work')

work_interfere_vs_remote_bar

:Bars   [remote_work,Work Interference]   (Proportion)

In [29]:
count_df = df.groupby(['remote_work', 'mental_health_consequence']).size().reset_index(name='count')

# Calculate total counts for each 'remote_work' category, then proportions, so count of remote vs. non remote isn't considered.
total_counts = count_df.groupby('remote_work')['count'].transform('sum')
count_df['proportion'] = count_df['count'] / total_counts

relative_freq_bar = count_df.hvplot.bar(x='remote_work', y='proportion', by='mental_health_consequence', stacked=False, height=400, width=800, xlabel='Remote Work', ylabel='Proportion', rot=45, title='Relative Frequency Between Mental Health Consequence and Remote Work')

relative_freq_bar

:Bars   [remote_work,mental_health_consequence]   (proportion)

In [59]:
dashboard = pn.Column(
    "# Survey Dashboard",
    pn.Row(
        pn.Column(bar_chart),
        pn.Column(plot2)
    ),
    pn.Row(
        pn.Column(relative_freq_bar),
        pn.Column(work_interfere_vs_remote_bar)
    ),
    pn.Row(bokeh_pane, bokeh_pane2),
    pn.Row(
        pn.Column(plot),
        pn.Column(box_plot)
    ),
    sizing_mode='stretch_width'
)

# Show the dashboard in the notebook
dashboard.servable()

Column(sizing_mode='stretch_width')
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] HoloViews(Bars, height=400, sizing_mode='fixed', width=1000)
        [1] Column
            [0] HoloViews(Bars, height=400, sizing_mode='fixed', width=800)
    [2] Row
        [0] Column
            [0] HoloViews(Bars, height=400, sizing_mode='fixed', width=800)
        [1] Column
            [0] HoloViews(Bars, height=400, sizing_mode='fixed', width=800)
    [3] Row
        [0] Bokeh(figure, theme='dark_minimal')
        [1] Bokeh(figure, theme='dark_minimal')
    [4] Row
        [0] Column
            [0] HoloViews(Bars, height=400, sizing_mode='fixed', width=800)
        [1] Column
            [0] HoloViews(Overlay, height=400, sizing_mode='fixed', width=1000)